# Trending | YouTube and Billboard

In [1]:
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pprint import pprint
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pymysql

## E X T R A C T

### Extract Youtube CSV into DataFrame

In [3]:
youtube_file = "./USvideos.csv"
youtubecsv_df = pd.read_csv(youtube_file)
youtubecsv_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,2kyS6SvSYSE,11/14/2017,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,...,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,NaN,NaN,NaN,NaN,NaN
1,1ZAPwfrtAFY,11/14/2017,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,...,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",NaN,NaN,NaN,NaN,NaN
2,5qpjK5DgCt4,11/14/2017,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,...,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,NaN,NaN,NaN,NaN,NaN
3,puqaWrEC7tY,11/14/2017,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,...,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,NaN,NaN,NaN,NaN,NaN
4,d380meD0W0M,11/14/2017,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,...,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,NaN,NaN,NaN,NaN,NaN


### Import YouTube JSON

In [4]:
# import JSON file
with open('US_category_id.json') as json_file:
    json_data = json.load(json_file)

pprint(json_data)

{'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'items': [{'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
            'id': '1',
            'kind': 'youtube#videoCategory',
            'snippet': {'assignable': True,
                        'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                        'title': 'Film & Animation'}},
           {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
            'id': '2',
            'kind': 'youtube#videoCategory',
            'snippet': {'assignable': True,
                        'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                        'title': 'Autos & Vehicles'}},
           {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
            'id': '10',
            'kind': 'youtube#videoCategory',
            'snippet': {'assignable': True,
                        'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                        'title': 'Music'}},
 

### Billboard import

In [5]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.billboard.com/charts/hot-100/2017-11-14'
browser.visit(url)

## T R A N S F O R M

### YouTube JSON to DataFrame

In [6]:
# Navigating JSON file to find category id/title
title = json_data["items"][0]["snippet"]["title"]
id = json_data["items"][0]["id"]
print(f'{id}, {title}')

1, Film & Animation


In [7]:
# loop through JSON to get list of ids & categories
items = json_data["items"]
ids = []
titles = []
for item in items:
    ids.append(item["id"])
    titles.append(item["snippet"]["title"])
titles

['Film & Animation',
 'Autos & Vehicles',
 'Music',
 'Pets & Animals',
 'Sports',
 'Short Movies',
 'Travel & Events',
 'Gaming',
 'Videoblogging',
 'People & Blogs',
 'Comedy',
 'Entertainment',
 'News & Politics',
 'Howto & Style',
 'Education',
 'Science & Technology',
 'Nonprofits & Activism',
 'Movies',
 'Anime/Animation',
 'Action/Adventure',
 'Classics',
 'Comedy',
 'Documentary',
 'Drama',
 'Family',
 'Foreign',
 'Horror',
 'Sci-Fi/Fantasy',
 'Thriller',
 'Shorts',
 'Shows',
 'Trailers']

In [9]:
category_list = pd.DataFrame(
    {'cat_id': ids,
     'category': titles
    })
category_list

,cat_id,category
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


### Merge Youtube CSV and JSON in DataFrame

In [10]:
youtubecsv_df = youtubecsv_df.rename(columns = {"category_id": "cat_id"})
youtubecsv_df.dtypes
category_list.dtypes
category_list["cat_id"] = category_list['cat_id'].astype(str).astype(int)
category_list.dtypes

cat_id       int32
category    object
dtype: object

In [11]:
merged_df = pd.merge(youtubecsv_df, category_list, how = "left", on="cat_id")
merged_df.head()

,video_id,trending_date,title,channel_title,cat_id,publish_time,tags,views,likes,dislikes,...,comments_disabled,ratings_disabled,video_error_or_removed,description,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,category
0,2kyS6SvSYSE,11/14/2017,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,...,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,NaN,NaN,NaN,NaN,NaN,People & Blogs
1,1ZAPwfrtAFY,11/14/2017,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,...,False,False,False,"One year after the presidential election, John...",NaN,NaN,NaN,NaN,NaN,Entertainment
2,5qpjK5DgCt4,11/14/2017,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,...,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,NaN,NaN,NaN,NaN,NaN,Comedy
3,puqaWrEC7tY,11/14/2017,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,...,False,False,False,Today we find out if Link is a Nickelback amat...,NaN,NaN,NaN,NaN,NaN,Entertainment
4,d380meD0W0M,11/14/2017,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,...,False,False,False,I know it's been a while since we did this sho...,NaN,NaN,NaN,NaN,NaN,Entertainment


### Transformed YouTube Dataframe(s)

In [36]:
# Create a filtered dataframe from specific columns
youtube_cols = ["video_id", "trending_date", "cat_id", "views", "likes", "dislikes", "comment_count", "category"]
youtube_df = merged_df[youtube_cols].copy()
youtube_df["trending_date"] = youtube_df["trending_date"].astype("datetime64")

youtube_df.head()

,video_id,trending_date,cat_id,views,likes,dislikes,comment_count,category
0,2kyS6SvSYSE,2017-11-14,22,748374,57527,2966,15954,People & Blogs
1,1ZAPwfrtAFY,2017-11-14,24,2418783,97185,6146,12703,Entertainment
2,5qpjK5DgCt4,2017-11-14,23,3191434,146033,5339,8181,Comedy
3,puqaWrEC7tY,2017-11-14,24,343168,10172,666,2146,Entertainment
4,d380meD0W0M,2017-11-14,24,2095731,132235,1989,17518,Entertainment


In [37]:
pd.to_datetime(youtube_df["trending_date"])
youtube_df.dtypes

video_id                 object
trending_date    datetime64[ns]
cat_id                    int64
views                     int64
likes                     int64
dislikes                  int64
comment_count             int64
category                 object
dtype: object

In [38]:
youtube_111417 = youtube_df.loc[youtube_df["trending_date"] == "11/14/2017"]

youtube_111417.dtypes

video_id                 object
trending_date    datetime64[ns]
cat_id                    int64
views                     int64
likes                     int64
dislikes                  int64
comment_count             int64
category                 object
dtype: object

### Billboard Soup

In [15]:
html = browser.html
soup = bs(html, 'html.parser')

In [16]:
artist_list=[]
song_list=[]

In [17]:
top_song = soup.find('div', class_='chart-number-one__title').text.strip()
top_artist = soup.find('div', class_='chart-number-one__artist').text.strip()
artist_list.append(top_artist)
song_list.append(top_song)

In [18]:
results = soup.find_all('div',class_='chart-list-item')

In [19]:
for x in results:
    artist_name = x.find('div', class_='chart-list-item__artist').text
    artist_list.append(artist_name.strip())
    song_title=x.find('span', class_='chart-list-item__title-text').text
    song_list.append(song_title.strip())

### Billboard DataFrame

In [46]:
Top_100_df=pd.DataFrame({'Artist':artist_list,'Song_Title':song_list})

Top_100_df.head()

,Artist,Song_Title
0,Post Malone Featuring 21 Savage,Rockstar
1,Cardi B,Bodak Yellow (Money Moves)
2,Logic Featuring Alessia Cara & Khalid,1-800-273-8255
3,Portugal. The Man,Feel It Still
4,Imagine Dragons,Thunder


## L O A D

### Create database connection

In [42]:
connection_string = "root:toor2@localhost/trending_db"
engine = create_engine(f'mysql://{connection_string}')

In [44]:
# Confirm tables
engine.table_names()

['billboard_table', 'trending_table', 'trending_table_111417']

### Load DataFrames into database

In [33]:
youtube_111417.iloc[119]

video_id                 _dhneCO4YEE
trending_date    2017-11-14 00:00:00
cat_id                            22
views                         362009
likes                           5505
dislikes                        2356
comment_count                   1264
category              People & Blogs
Name: 119, dtype: object

In [34]:
youtube_111417.to_sql(name='trending_table_111417', con=engine, if_exists='append', index=False)

In [39]:
youtube_df.to_sql(name='trending_table', con=engine, if_exists='append', index=False)

In [47]:
Top_100_df.to_sql(name='billboard_table', con=engine, if_exists='append', index=False)